---

**In this notebook, we will use Ensembling Gradient Boosting trees for a multi-target regression problem, leveraging lagged targets to predict 424 outputs. To speed up inference, we adopt the long-format multi-output prediction method, which is much faster than the standard multi-output approach.**

**You will also find several useful techniques in this notebook, including:**

* How to create lagged targets and use them in the prediction step
* How to run Ensembling Gradient Boosting trees with lags targets
* How to build an optimized prediction function for API inference



---

In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import warnings
warnings.filterwarnings("ignore")

# Data loading
p = '/kaggle/input/mitsui-commodity-prediction-challenge/'
train = pd.read_csv(p+'train.csv')
trainl = pd.read_csv(p+'train_labels.csv')
traint = pd.read_csv(p+'target_pairs.csv')

def _handle_missing_values(data):
    data.interpolate(method='polynomial', order=3, inplace=True)
    data.clip(lower=-10, upper=10, inplace=True)
    return data

train = _handle_missing_values(train)
trainl = _handle_missing_values(trainl)

target_lag_1 = traint.loc[traint["lag"]==1,"target"].values
target_lag_2 = traint.loc[traint["lag"]==2,"target"].values
target_lag_3 = traint.loc[traint["lag"]==3,"target"].values
target_lag_4 = traint.loc[traint["lag"]==4,"target"].values

Features = [i for i in trainl.columns]

def create_lagged_labels(df):
    dt = pd.DataFrame()
    dt["date_id"] = df["date_id"]
    for f in Features[1:]:
        if f in target_lag_1:
            lag = 1
        elif f in target_lag_2:
            lag = 2
        elif f in target_lag_3:
            lag = 3
        elif f in target_lag_4:
            lag = 4    
        dt[f] = df[f].shift(lag).fillna(0)
    return df, dt

_, train_lagged = create_lagged_labels(trainl)

# Create training data
import gc
training_df = []
target_cols = [f"target_{i}" for i in range(424)]
for j, target_col in enumerate(target_cols):
    temp_train_df = pd.DataFrame()
    temp_train_df[Features] = train_lagged[Features]                     
    temp_train_df['target_id'] = j
    y = trainl[target_col].values
    temp_train_df['target'] = y
    mask = ~(np.isnan(y) | np.isinf(y) | (np.abs(y) > 1e10))
    training_df.append(temp_train_df[mask].copy())
    del temp_train_df, y
    gc.collect()

training_df = pd.concat(training_df).reset_index(drop=True)
Features2 = Features + ["target_id"]
X_train = training_df[Features2]
y_train = training_df["target"]

Data preparation completed!
Training data shape: (831129, 426)
Features: ['date_id', 'target_0', 'target_1', 'target_2', 'target_3', 'target_4', 'target_5', 'target_6', 'target_7', 'target_8', 'target_9', 'target_10', 'target_11', 'target_12', 'target_13', 'target_14', 'target_15', 'target_16', 'target_17', 'target_18', 'target_19', 'target_20', 'target_21', 'target_22', 'target_23', 'target_24', 'target_25', 'target_26', 'target_27', 'target_28', 'target_29', 'target_30', 'target_31', 'target_32', 'target_33', 'target_34', 'target_35', 'target_36', 'target_37', 'target_38', 'target_39', 'target_40', 'target_41', 'target_42', 'target_43', 'target_44', 'target_45', 'target_46', 'target_47', 'target_48', 'target_49', 'target_50', 'target_51', 'target_52', 'target_53', 'target_54', 'target_55', 'target_56', 'target_57', 'target_58', 'target_59', 'target_60', 'target_61', 'target_62', 'target_63', 'target_64', 'target_65', 'target_66', 'target_67', 'target_68', 'target_69', 'target_70', 't

In [2]:
# Deeper Gradient Boosting models for regression
xgb_model = XGBRegressor(
    n_estimators=2000,
    max_depth=6,
    learning_rate=0.01,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=1,
    reg_lambda=1,
    random_state=42,
    tree_method="hist",
    device="cuda"
)

lgbm_model = LGBMRegressor(
    n_estimators=2000,
    max_depth=6,
    learning_rate=0.01,
    num_leaves=256,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=1,
    reg_lambda=1,
    random_state=42,
    device="gpu",
    verbose=-1
)

catboost_model = CatBoostRegressor(
    iterations=2000,
    depth=6,
    learning_rate=0.01,
    l2_leaf_reg=3,
    random_seed=42,
    loss_function='RMSE',
    task_type="GPU",
    verbose=False
)

# Append models to a list for later training / ensembling
models = [xgb_model, lgbm_model, catboost_model]
Models = []

# Train all models on the entire dataset (no target-specific training)
print("Training models on entire dataset...")
for model in models:
    model.fit(X_train, y_train)
    Models.append(model)

print(f"Models list created with {len(Models)} models.")

In [6]:
def ensemble_predict(models, X):
    """
    Predict using a list of trained models and return the averaged prediction.
    
    Parameters:
        models : list of trained models
        X      : numpy array or DataFrame of features
        
    Returns:
        ensemble_pred : averaged prediction across models
    """
    preds = [model.predict(X) for model in models]
    ensemble_pred = np.mean(preds, axis=0)
    return ensemble_pred

# Test the predictions
X_data = X_train.copy()
X_data["preds"] = ensemble_predict(Models, X_train)

# Convert to wide format (90 rows × 424 columns)
df_preds = X_data.copy()
df_preds['row'] = df_preds.groupby('target_id').cumcount()

# Pivot the table to wide format
df_wide = df_preds.pivot(index='row', columns='target_id', values='preds')
df_wide = df_wide.sort_index(axis=1)
df_wide.index = [i for i in df_wide.index]

# Rename columns
df_wide.columns = [f'target_{i}' for i in df_wide.columns]
print(f"Wide format shape: {df_wide.shape}")

In [10]:
def rank_correlation_sharpe_ratio(merged_df: pd.DataFrame) -> float:
    prediction_cols = [col for col in merged_df.columns if col.startswith('prediction_')]
    target_cols = [col for col in merged_df.columns if col.startswith('target_')]
    
    def _compute_rank_correlation(row):
        non_null_targets = [col for col in target_cols if not pd.isnull(row[col])]
        matching_predictions = [col for col in prediction_cols if col.replace('prediction', 'target') in non_null_targets]
        
        if not non_null_targets:
            raise ValueError('No non-null target values found')
        if row[non_null_targets].std(ddof=0) == 0 or row[matching_predictions].std(ddof=0) == 0:
            raise ZeroDivisionError('Denominator is zero, unable to compute rank correlation.')
        
        return np.corrcoef(
            row[matching_predictions].rank(method='average'), 
            row[non_null_targets].rank(method='average')
        )[0, 1]
    
    daily_rank_corrs = merged_df.apply(_compute_rank_correlation, axis=1)
    std_dev = daily_rank_corrs.std(ddof=0)
    
    if std_dev == 0:
        raise ZeroDivisionError('Denominator is zero, unable to compute Sharpe ratio.')
    
    sharpe_ratio = daily_rank_corrs.mean() / std_dev
    return float(sharpe_ratio)

def score(solution: pd.DataFrame, submission: pd.DataFrame) -> float:
    assert all(solution.columns == submission.columns)
    submission = submission.rename(columns={col: col.replace('target_', 'prediction_') for col in submission.columns})
    solution = solution.replace(0, None)
    return rank_correlation_sharpe_ratio(pd.concat([solution, submission], axis='columns'))

# Test scoring
score_value = score(trainl[Features[1:]], df_wide[Features[1:]])
print(f"SCORE: {score_value:.6f}")

In [12]:
import polars as pl

def predict(
    test: pl.DataFrame,
    lag1: pl.DataFrame, 
    lag2: pl.DataFrame,
    lag3: pl.DataFrame,
    lag4: pl.DataFrame,
) -> pl.DataFrame:
    """
    Predicts target values using lag features.
    This is your working version from the notebook.
    """
    # Convert to pandas
    test_pd = test.to_pandas()
    lag1_pd = lag1.to_pandas()
    lag2_pd = lag2.to_pandas()
    lag3_pd = lag3.to_pandas()
    lag4_pd = lag4.to_pandas()
    
    # Combine lag features
    X_pred = pd.concat([
        test_pd[["date_id"]],
        lag1_pd[target_lag_1],
        lag2_pd[target_lag_2],
        lag3_pd[target_lag_3],
        lag4_pd[target_lag_4],
    ], axis=1)
    
    # If no rows, return all zeros
    if X_pred.height == 0:
        return pl.DataFrame(0, schema=[(f"target_{i}", pl.Float64) for i in range(424)])
    
    # Fill nulls with 0
    X_pred = X_pred.fillna(0)
    
    # Prepare features for prediction
    n_targets = 424
    n_rows = X_pred.shape[0]
    
    # Create features for all targets
    features_array = np.tile(X_pred[Features[1:]].values, (n_targets, 1))
    target_ids = np.repeat(np.arange(n_targets), n_rows)
    
    # Create prediction DataFrame
    X_pred2 = pd.DataFrame({
        "date_id": np.tile(X_pred["date_id"].values, n_targets),
        **{feat: features_array[:, i] for i, feat in enumerate(Features[1:])},
        "target_id": target_ids,
        "row": np.tile(np.arange(n_rows), n_targets)
    })
    
    # Make predictions
    preds = ensemble_predict(Models, X_pred2[Features2])
    X_pred2 = X_pred2.assign(preds=preds)
    
    # Pivot to wide format
    df_wide = (
        X_pred2.groupby(["target_id", "row"])
        .agg({"preds": "first"})
        .reset_index()
        .pivot(index="row", columns="target_id", values="preds")
        .sort_index()
    )
    
    # Ensure correct column order
    df_wide = df_wide.reindex(columns=range(424), fill_value=0)
    df_wide.columns = [f"target_{i}" for i in range(424)]
    
    # Return last row as predictions
    result_df = df_wide.tail(1)
    return pl.from_pandas(result_df.reset_index(drop=True))

# Test the prediction function
def test_prediction():
    """Test the prediction function"""
    sample_test = pl.from_pandas(trainl[Features].iloc[:5])
    sample_lag1 = pl.from_pandas(trainl[target_lag_1].iloc[:5])
    sample_lag2 = pl.from_pandas(trainl[target_lag_2].iloc[:5])
    sample_lag3 = pl.from_pandas(trainl[target_lag_3].iloc[:5])
    sample_lag4 = pl.from_pandas(trainl[target_lag_4].iloc[:5])
    
    result = predict(sample_test, sample_lag1, sample_lag2, sample_lag3, sample_lag4)
    print(f"Prediction result shape: {result.shape}")
    return result

test_result = test_prediction()

Save/load functions ready!


In [11]:
import joblib

def save_models():
    """Save the trained models"""
    joblib.dump(Models, '/kaggle/working/models_list.joblib')
    print(f"Saved {len(Models)} models")

def load_models():
    """Load the trained models"""
    Models = joblib.load('/kaggle/working/models_list.joblib')
    print(f"Loaded {len(Models)} models")
    return Models

# Save models
save_models()

# For competition submission
import os
import kaggle_evaluation.mitsui_inference_server

inference_server = kaggle_evaluation.mitsui_inference_server.MitsuiInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(('/kaggle/input/mitsui-commodity-prediction-challenge/',))

=== TESTING POLARS PREDICTION ===
Input types:
test: <class 'polars.dataframe.frame.DataFrame'>
lag1: <class 'polars.dataframe.frame.DataFrame'>
lag2: <class 'polars.dataframe.frame.DataFrame'>
lag3: <class 'polars.dataframe.frame.DataFrame'>
lag4: <class 'polars.dataframe.frame.DataFrame'>
Input shapes - test: (5, 425), lag1: (5, 106), lag2: (5, 106), lag3: (5, 106), lag4: (5, 106)
Models list loaded from /kaggle/input/mitsuienslearning/models_list.joblib with 1272 models
Loaded 1272 models for prediction
Creating features for 5 rows and 424 targets...
Prediction DataFrame shape: (2120, 427)
Predictions completed, min: -0.0002, max: 0.0000, mean: -0.0002
Wide format shape: (5, 424)
Final result shape: (1, 424)
Result type: <class 'polars.dataframe.frame.DataFrame'>
Result shape: (1, 424)
Result columns: ['target_0', 'target_1', 'target_2', 'target_3', 'target_4', 'target_5', 'target_6', 'target_7', 'target_8', 'target_9']...
Prediction block ready! Use 'competition_predict' for submis

In [ ]:
# submission through the API
import os
import kaggle_evaluation.mitsui_inference_server

inference_server = kaggle_evaluation.mitsui_inference_server.MitsuiInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    print('there')
    # inference_server.serve()
else:
    print('here')
    inference_server.run_local_gateway(('/kaggle/input/mitsui-commodity-prediction-challenge/',))

In [16]:
display(pl.read_parquet('/kaggle/working/submission.parquet'))

date_id,target_0,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,target_9,target_10,target_11,target_12,target_13,target_14,target_15,target_16,target_17,target_18,target_19,target_20,target_21,target_22,target_23,target_24,target_25,target_26,target_27,target_28,target_29,target_30,target_31,target_32,target_33,target_34,target_35,…,target_387,target_388,target_389,target_390,target_391,target_392,target_393,target_394,target_395,target_396,target_397,target_398,target_399,target_400,target_401,target_402,target_403,target_404,target_405,target_406,target_407,target_408,target_409,target_410,target_411,target_412,target_413,target_414,target_415,target_416,target_417,target_418,target_419,target_420,target_421,target_422,target_423
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1827,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,…,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068
1828,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,…,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068,-0.000068
1829,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,…,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136,-0.000136
1830,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,…,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.000235,-0.0002